# Time Windows

This notebook covers the best practices for creating time windows from a given dataset for the purpose of time series forcasting, and demonstrates a number of useful data processing techniques with numpy and the built in lambda function.

## Setup

In [ ]:
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

## Time Windows

In [ ]:
# make dataset of range(10)
dataset = 
for val in dataset:
    print(val.numpy())

In [ ]:
# with windows
dataset = 
dataset = 
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

In [ ]:
# no remainders
dataset = 
dataset = 
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

In [ ]:
# batch windows
dataset = 
dataset = 
dataset = 
for window in dataset:
    print(window.numpy())

In [ ]:
# split label from features
dataset = 
dataset = 
dataset = 
dataset = 
for x, y in dataset:
    print(x.numpy(), y.numpy())

In [ ]:
# shuffle 
dataset =  
dataset =  
dataset =  
dataset =  
dataset =  
for x, y in dataset:
    print(x.numpy(), y.numpy())

In [ ]:
# batch and prefetch
dataset =  
dataset =  
dataset =  
dataset =  
dataset =  
dataset =  
for x, y in dataset:
    print("x =", x.numpy())
    print("y =", y.numpy())

In [ ]:
# all together (useful function)
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    
    return dataset